<a href="https://colab.research.google.com/github/yisakberhanu/AI-Mathematical-Olympiad/blob/main/generate_strategy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Generate Strategy

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
def load_data(grid_outage_file, solar_prediction_file, load_prediction_file):
    grid_outage = pd.read_csv(grid_outage_file)
    solar_prediction = pd.read_csv(solar_prediction_file)
    load_prediction = pd.read_csv(load_prediction_file)
    return grid_outage, solar_prediction, load_prediction
def calculate_soc(current_soc, power_delta, battery_capacity, charge_coef, discharge_coef):
    if power_delta > 0:  # Charging
        soc_delta = (power_delta * charge_coef) / battery_capacity
    else:  # Discharging
        soc_delta = (power_delta * discharge_coef) / battery_capacity
    new_soc = current_soc + soc_delta
    return max(0, min(1, new_soc))  # Ensure SOC is between 0 and 1

In [ ]:
site_config, solar_prediction, load_prediction = load_data('/kaggle/input/smart-energy/smart-energy-supply-scheduling-for-green-telecom20240806-1019-x8g4w/trainData/trainData/site information and grid outage.csv',
                                                           '/kaggle/input/smart-energy/solar_test_preds.csv',
                                                           '/kaggle/input/smart-energy/energy_test_preds.csv')
display(site_config.head(), solar_prediction.head(), load_prediction.head())

,Site Id,diesel power(kW),battery capacity (Ah),rated voltage（V）,battery charge coefficient,battery discharge coefficient,init SOC,DOD,grid power(kW),grid outage plan
0,site1,18.0,300,54,0.35,1.2,0.2,0.15,8,[false false false false false false false fal...
1,site2,17.6,760,54,0.35,1.2,0.2,0.15,14,[false false false false false false false fal...
2,site3,12.8,760,54,0.35,1.2,0.2,0.15,10,[false false false false false false false fal...
3,site4,17.6,760,54,0.35,1.2,0.2,0.15,14,[true true true true true true true true true ...
4,site5,17.6,380,54,0.35,1.2,0.2,0.15,12,[false false false false false true true true ...


,Site Name,Day,Hour,solar_preds
0,site1,61,0,-0.017261
1,site10,61,0,0.000283
2,site2,61,0,-0.028670
3,site3,61,0,-0.002697
4,site4,61,0,-0.009436


,Site Name,Day,Hour,energy_preds
0,site1,61,0,3.694253
1,site10,61,0,4.034325
2,site2,61,0,8.774539
3,site3,61,0,1.843485
4,site4,61,0,8.929755


In [ ]:
solar_prediction['solar_preds'] = np.where(solar_prediction['solar_preds']<0, 0, solar_prediction['solar_preds'])

In [ ]:
string = "[false,false,false,false,false,false]"
boolean_list = eval(string.replace('false', 'False'))
print(boolean_list)


[False, False, False, False, False, False]


In [ ]:
site_config['grid outage plan'] = site_config['grid outage plan'].str.replace(" ", ",")
display(site_config.head())

,Site Id,diesel power(kW),battery capacity (Ah),rated voltage（V）,battery charge coefficient,battery discharge coefficient,init SOC,DOD,grid power(kW),grid outage plan
0,site1,18.0,300,54,0.35,1.2,0.2,0.15,8,"[false,false,false,false,false,false,false,fal..."
1,site2,17.6,760,54,0.35,1.2,0.2,0.15,14,"[false,false,false,false,false,false,false,fal..."
2,site3,12.8,760,54,0.35,1.2,0.2,0.15,10,"[false,false,false,false,false,false,false,fal..."
3,site4,17.6,760,54,0.35,1.2,0.2,0.15,14,"[true,true,true,true,true,true,true,true,true,..."
4,site5,17.6,380,54,0.35,1.2,0.2,0.15,12,"[false,false,false,false,false,true,true,true,..."


In [ ]:
site_config.columns

Index(['Site Id', 'diesel power(kW)', 'battery capacity (Ah)',
       'rated voltage（V）', 'battery charge coefficient',
       'battery discharge coefficient', 'init SOC', 'DOD', 'grid power(kW)',
       'grid outage plan'],
      dtype='object')

In [ ]:
grid_outage = pd.DataFrame()
for i, row in site_config.iterrows():
    site_name = row['Site Id']
    outage_plan = eval(row['grid outage plan'].replace('true', 'True').replace('false', 'False'))
    assert len(outage_plan) == 168

    # Create a DataFrame for the current site with outage plan for each hour
    site_df = pd.DataFrame({'Site Id': site_name, 'hour': range(1, 169), 'grid_outage': outage_plan})

    grid_outage = pd.concat([grid_outage, site_df], ignore_index=True)

site_config.drop(columns=['grid outage plan'], inplace=True)
display(grid_outage.head())

,Site Id,hour,grid_outage
0,site1,1,False
1,site1,2,False
2,site1,3,False
3,site1,4,False
4,site1,5,False


In [ ]:

# Add 'daily_hr' column (0-23 repeated)
grid_outage['daily_hr'] = (grid_outage['hour'] - 1) % 24

# Add 'day' column (61 to 67 repeated for each 24 hours)
grid_outage['day'] = 61 + (grid_outage['hour'] - 1) // 24
display(grid_outage.head())

,Site Id,hour,grid_outage,daily_hr,day
0,site1,1,False,0,61
1,site1,2,False,1,61
2,site1,3,False,2,61
3,site1,4,False,3,61
4,site1,5,False,4,61


In [ ]:


def calculate_soc(current_soc, power_delta, battery_capacity, charge_coef, discharge_coef):
    if power_delta > 0:  # Charging
        soc_delta = (power_delta * charge_coef) / battery_capacity
    else:  # Discharging
        soc_delta = (power_delta * discharge_coef) / battery_capacity
    new_soc = current_soc + soc_delta
    return max(0, min(1, new_soc))  # Ensure SOC is between 0 and 1

def generate_day_night_battery_strategy(site_config, solar_pred, load_pred, grid_outage_plan):
    strategy = []
    current_soc = site_config['init SOC'].values[0]
    battery_capacity = site_config['battery capacity (Ah)'].values[0] * site_config['rated voltage（V）'].values[0] / 1000  # Convert to kWh
    DOD = site_config['DOD'].values[0]
    min_soc = 1 - DOD  # Calculate the minimum allowable SOC
    charge_coeff = site_config['battery charge coefficient'].values[0]
    discharge_coeff = site_config['battery discharge coefficient'].values[0]

    for day in range(61, 68):
        solar_available_day = solar_pred[solar_pred['Day'] == day]
        load_required_day = load_pred[load_pred['Day'] == day]
        grid_available_day = grid_outage_plan[grid_outage_plan['day'] == day]

        for hour in range(24):
            is_daytime = 6 <= hour < 18  # Assume daytime is from 6 AM to 6 PM
            for quarter in range(4):  # 15-minute intervals
                solar_available = solar_available_day[solar_available_day['Hour'] == hour]['solar_preds'].values[0] / 4
                load_required = load_required_day[load_required_day['Hour'] == hour]['energy_preds'].values[0] / 4
                grid_available = grid_available_day[grid_available_day['daily_hr'] == hour]['grid_outage'].values[0]

                use_solar = solar_available > 0
                use_grid = False
                use_diesel = False

                if is_daytime:
                    # During the day, prioritize charging the battery
                    if use_solar:
                        battery_charge_capacity = (1 - current_soc) * battery_capacity
                        power_to_battery = min(solar_available, battery_charge_capacity)
                        current_soc = calculate_soc(current_soc, power_to_battery, battery_capacity, charge_coeff, discharge_coeff)

                        # Use remaining solar power for load
                        remaining_solar = solar_available - power_to_battery
                        if remaining_solar >= load_required:
                            power_balance = 0
                        else:
                            power_balance = remaining_solar - load_required
                    else:
                        power_balance = -load_required
                else:
                    # During the night, use battery first
                    power_balance = -load_required
                    battery_discharge_capacity = (current_soc - min_soc) * battery_capacity
                    if battery_discharge_capacity > 0:
                        power_from_battery = min(load_required, battery_discharge_capacity)
                        new_soc = calculate_soc(current_soc, -power_from_battery, battery_capacity, charge_coeff, discharge_coeff)
                        if new_soc >= min_soc:
                            power_balance += power_from_battery
                            current_soc = new_soc

                # If there's still a power deficit, use grid if available
                if power_balance < 0 and grid_available:
                    use_grid = True
                    power_balance = 0

                # If there's still a power deficit, use diesel as last resort
                if power_balance < 0:
                    use_diesel = True
                    power_balance = 0

                strategy.append({
                    'use_solar': use_solar,
                    'use_grid': use_grid,
                    'use_diesel': use_diesel,
                    'current_soc': current_soc
                })

    return strategy

# Main execution
all_site_strategies = {}
for site in site_config['Site Id'].unique():
    site_configuration = site_config[site_config['Site Id'] == site]
    solar_pred = solar_prediction[solar_prediction['Site Name'] == site]
    energy_pred = load_prediction[load_prediction['Site Name'] == site]
    grid_outage_plan = grid_outage[grid_outage['Site Id'] == site]
    strategy = generate_day_night_battery_strategy(site_config=site_configuration,
                                                   solar_pred=solar_pred,
                                                   load_pred=energy_pred,
                                                   grid_outage_plan=grid_outage_plan)
    all_site_strategies[site] = strategy

# Convert strategies to the required output format
output_data = []
for site_id, strategy in all_site_strategies.items():
    for time_step, decision in enumerate(strategy):
        output_data.append({
            'site': site_id,
            'hour': time_step,
            'use_grid': decision['use_grid'],
            'use_diesel': decision['use_diesel'],
            'use_solar': decision['use_solar']
        })

output_df = pd.DataFrame(output_data)
print(output_df.shape)
output_df.to_csv('strategy_output.csv', index=False)

# Print statistics for each site
for site_id, strategy in all_site_strategies.items():
    min_soc = min(decision['current_soc'] for decision in strategy)
    avg_soc = sum(decision['current_soc'] for decision in strategy) / len(strategy)
    solar_usage = sum(1 for decision in strategy if decision['use_solar'])
    grid_usage = sum(1 for decision in strategy if decision['use_grid'])
    diesel_usage = sum(1 for decision in strategy if decision['use_diesel'])

    print(f"Site {site_id}:")
    print(f"  Minimum SOC: {min_soc:.4f}")
    print(f"  Average SOC: {avg_soc:.4f}")
    print(f"  Solar usage: {solar_usage} time steps")
    print(f"  Grid usage: {grid_usage} time steps")
    print(f"  Diesel usage: {diesel_usage} time steps")
    print()

(6720, 5)
Site site1:
  Minimum SOC: 0.2000
  Average SOC: 0.8963
  Solar usage: 560 time steps
  Grid usage: 93 time steps
  Diesel usage: 436 time steps

Site site2:
  Minimum SOC: 0.2000
  Average SOC: 0.8410
  Solar usage: 420 time steps
  Grid usage: 250 time steps
  Diesel usage: 410 time steps

Site site3:
  Minimum SOC: 0.2000
  Average SOC: 0.8661
  Solar usage: 504 time steps
  Grid usage: 210 time steps
  Diesel usage: 354 time steps

Site site4:
  Minimum SOC: 0.2000
  Average SOC: 0.8740
  Solar usage: 512 time steps
  Grid usage: 160 time steps
  Diesel usage: 355 time steps

Site site5:
  Minimum SOC: 0.2000
  Average SOC: 0.8793
  Solar usage: 556 time steps
  Grid usage: 303 time steps
  Diesel usage: 350 time steps

Site site6:
  Minimum SOC: 0.2000
  Average SOC: 0.8074
  Solar usage: 496 time steps
  Grid usage: 113 time steps
  Diesel usage: 514 time steps

Site site7:
  Minimum SOC: 0.2000
  Average SOC: 0.8789
  Solar usage: 504 time steps
  Grid usage: 218 time 